# My Recommended Vacation Destinations
----
##### Based on analysis of the weather on 8/12/2022, I have provided a recomendation of possible vacation locations (with hotel) meeting my 'ideal' weather criteria.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import warnings
warnings.filterwarnings('ignore')

# Import API key
from api_keys import g_key

In [2]:
city_data = pd.read_csv("output_folder/cities.csv")
city_data.head(5)

,City,Lat,Lng,Min_Temp,Max_Temp,Humidity,Cloudiness,Wind_Speed,Country,Date
0,New Norfolk,-42.7826,147.0587,53.10,56.71,78,98,1.41,AU,1660363485
1,Batagay,67.6333,134.6333,59.18,59.18,54,80,10.92,RU,1660363485
2,Tāttayyangārpettai,11.1333,78.4500,85.35,85.35,56,83,8.05,IN,1660363486
3,Bredasdorp,-34.5322,20.0403,43.30,43.30,86,54,4.70,ZA,1660363487
4,Nantucket,41.2835,-70.0995,66.02,67.14,45,99,6.85,US,1660363487


### Humidity Heatmap

In [3]:
# Configure gmaps
gmaps.configure(api_key = g_key)

# Use the Lat and Lng as locations and Humidity as the weight.
locations = city_data[["Lat", "Lng"]]
humidity = city_data["Humidity"].astype(float)

# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights = humidity, max_intensity = 100)

# Add Heatmap layer to map
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### My Ideal locations based on weather
#### Criteria:
* Max temperature <= 80
* Min temperature >= 75
* Wind Speed < 7 mi/hr
* Cloudiness <= 50
* Humidity <= 75

In [4]:
#Narrow to my ideal conditions:
#Max temperature <= 80
#Min temperature >= 75
#Wind Speed < 7 mi/hr
#Cloudiness <= 50
#Humidity <= 75

ideal_cities = city_data.loc[(city_data['Max_Temp'] <= 80) & (city_data['Min_Temp'] >= 75) & (city_data['Wind_Speed'] < 7) & (city_data['Cloudiness'] <= 50) & (city_data['Humidity'] <= 75)]
ideal_cities

,City,Lat,Lng,Min_Temp,Max_Temp,Humidity,Cloudiness,Wind_Speed,Country,Date
32,Puerto Escondido,15.8500,-97.0667,79.63,79.63,65,40,5.75,MX,1660363501
43,Djebeniana,35.0350,10.9081,79.16,79.16,63,0,6.46,TN,1660363505
134,Dali,25.7000,100.1833,76.39,76.39,62,10,1.54,CN,1660363548
184,Airai,-8.9266,125.4092,75.33,75.33,44,9,3.69,TL,1660363569
211,Tiznit Province,29.5833,-9.5000,76.28,76.28,48,1,1.72,MA,1660363581
396,Jalu,29.0331,21.5482,77.81,77.81,37,0,1.83,LY,1660363660
494,Ayní,39.3941,68.5377,76.35,76.35,33,0,1.28,TJ,1660363700


### Hotel Map

In [5]:
#Store into variable named hotel_df
#Add a "Hotel Name" column to the DataFrame
hotel_df = ideal_cities
hotel_df["Hotel_Name"] = ""
hotel_df.reset_index(inplace = True)

In [6]:
counter = 0

#Loop through the dataframe to hit the Google Places API for each city's coordinates
for ind in hotel_df.index:
    
    #Set parameters to search for hotels with 5000 meters
    target_search = "hotel"
    coords = f"{hotel_df['Lat'][ind]},{hotel_df['Lng'][ind]}"
    radius = 5000
    
    # assign params dict
    params = {
        "location": coords,
        "keyword": target_search,
        "radius": radius,
        "key": g_key
    }

    # Build URL using the Google Maps API
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # Run request
    response = requests.get(base_url, params)

    #Store the first Hotel result into the DataFrame
    returned_hotel = response.json()
    
    try:
        #If there at least one hotel found, put the name in the table
        hotel_df['Hotel_Name'][ind] = returned_hotel["results"][0]["name"]
        
    except:
        #handle issue of no hotels found
        hotel_df['Hotel_Name'][ind] = "No Hotel Found - bring a tent!"
        pass
    
hotel_df

,index,City,Lat,Lng,Min_Temp,Max_Temp,Humidity,Cloudiness,Wind_Speed,Country,Date,Hotel_Name
0,32,Puerto Escondido,15.8500,-97.0667,79.63,79.63,65,40,5.75,MX,1660363501,Villas Carrizalillo Hotel
1,43,Djebeniana,35.0350,10.9081,79.16,79.16,63,0,6.46,TN,1660363505,No Hotel Found - bring a tent!
2,134,Dali,25.7000,100.1833,76.39,76.39,62,10,1.54,CN,1660363548,LUX* Tea Horse Road Dali
3,184,Airai,-8.9266,125.4092,75.33,75.33,44,9,3.69,TL,1660363569,No Hotel Found - bring a tent!
4,211,Tiznit Province,29.5833,-9.5000,76.28,76.28,48,1,1.72,MA,1660363581,No Hotel Found - bring a tent!
5,396,Jalu,29.0331,21.5482,77.81,77.81,37,0,1.83,LY,1660363660,إستراحة صلاح الصادق
6,494,Ayní,39.3941,68.5377,76.35,76.35,33,0,1.28,TJ,1660363700,Меҳмонхона СУҒД


In [7]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel_Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [8]:
#Plot markers on top of the heatmap
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(marker_layer)

# Display figure
fig


Figure(layout=FigureLayout(height='420px'))